# **Step 1° - Convolution of hyperspectral data**

In [ ]:
import os
import pandas as pd

# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")

# Reading the CSV file
DB_raw = pd.read_csv("DB_validation_raw_clustered.csv", encoding='ISO-8859-1')

hyper_data = DB_raw

# Select the ID column number and the numbers where the hyperspectral bands begin and end
hyper_data = hyper_data.iloc[:, [0] + list(range(16, 2167))]

In [ ]:
print(hyper_data)

In [ ]:
##################################################################
#  At this stage of the script, data from hyperspectral bands    #
#  to multispectral bands is convolved.                          #
#  Pay attention to the script comments at this stage            #
##################################################################


# Imports
# Se estiver executando o script na máquina pessoal, insira o diretório da linguagem R instalada na máquina
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.3.2'
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Saving Dataframes to CSV

os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
hyper_data.to_csv("hyper_data.csv", index=False)

# R Script
r_script = """

# Reading CSV files

# Ajuste do limite de memória no início do script
#memory.limit(size=20000000)

setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
hyper_data <- read.csv("hyper_data.csv", header = TRUE, sep = ",")

rownames(hyper_data) <- hyper_data[, 1]
hyper_data1 <- hyper_data
hyper_data1 <- hyper_data1[, -1]
ID_Unico <- hyper_data[, 1]
colnames(hyper_data1) <- seq(from = 350, to = 2500, by = 1)


##############################################################################################
## R: Directory path to the required hsdar package libraries##                               #
#                                                                                            #
.libPaths("G:/OneDrive/Documentos/Doutorado/protocol_database/libraires")                    #
#                                                                                            #
##############################################################################################

# Load the hsdar package
library(hsdar)


# Create a matrix of the data
hyper_data.matrix <- as.matrix(hyper_data1)

# Wavelengths for hyperspectral data
wave.hyper_data <- seq(from = 350, to = 2500, by = 1)

# Create speclib of the data
hyper_data.speclib <- speclib(hyper_data.matrix, wave.hyper_data)

# Get characteristics of the sensor
get.sensor.characteristics("Sentinel2a", response_function = TRUE)


############################################################################################
#The following command will obtain the channel wavelength of deployed
#satellite sensors (multispectral).
#Below is a list of sensor names available for operation.
#Copy and paste the name into the command:

#"Landsat4"
#"Landsat5"
#"Landsat7"
#"Landsat8"
#"Sentinel2a"
#"Sentinel2b"
#"RapidEye
#"WorldView2-8"
#"Quickbird"
#"WorldView2-4"

# Perform spectral resampling
multi_data.sentinel2.1 <- spectralResampling(hyper_data.speclib, "Sentinel2a",
                                            response_function = TRUE)

############################################################################################

# Plot a graph
plot(multi_data.sentinel2.1[1])

# Summary statistics
summary(multi_data.sentinel2.1)


# Round the values before transforming into a dataframe
resampled_spectra <- as.data.frame(multi_data.sentinel2.1)

#resampled_spectra <- round(as.data.frame(multi_data.sentinel2.1))

# Add IDs back
resampled_spectra <- cbind(ID_Unico, resampled_spectra)

# Write the dataframe to a CSV file
setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
write.csv(resampled_spectra, "resampled_spectra.csv", row.names = FALSE)

"""
# Executing the R script
robjects.r(r_script)

In [ ]:
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
# Reading the CSV file

resampled_spectra = pd.read_csv("resampled_spectra.csv")

print(resampled_spectra)

### **Data handling for Landsat 8 spectral resolution**

In [ ]:
# Landsat 8 spectral resolution:

# B1.........Costal...........430-450 (nm)
# B2.........Blue.............450-510 (nm)
# B3.........Green............530-590 (nm)
# B4.........Red..............640-670 (nm)
# B5.........NIR..............850-880 (nm)
# B9.........Cirrus.........1360-1380 (nm)
# B6.........SWIR1..........1570-1650 (nm)
# B7.........SWIR2..........2110-2290 (nm)


# Create a dictionary of correspondence between original and new names
corresponding_name = {
    "ID_Unico": "ID_Unico",
    "Costal": "Costal_Landsat8_resampled",
    "Blue": "Blue_Landsat8_resampled",
    "Green": "Green_Landsat8_resampled",
    "Red": "Red_Landsat8_resampled",
    "NIR": "NIR_Landsat8_resampled",
    "Cirrus": "Cirrus_Landsat8_resampled",
    "SWIR1": "SWIR1_Landsat8_resampled",
    "SWIR2": "SWIR2_Landsat8_resampled"
}

# Rename the columns of the resampled_spectra dataframe
resampled_spectra.columns = [corresponding_name[col] for col in resampled_spectra.columns]

# Merge the dataframes
DB_raw_resampled_spectra = pd.merge(DB_raw, resampled_spectra, on="ID_Unico")

In [ ]:
print(DB_raw_resampled_spectra)

In [ ]:
# Saving Dataframes to CSV
DB_raw_resampled_spectra.to_csv("DB_raw_resampled_spectra.csv", index=False)

### **Data handling for Sentinel-2A spectral resolution**

In [ ]:
# Sentinel-2A spectral resolution:
# B1.........Aerossol.........430-450 (nm)
# B2.........Blue.............458-523 (nm)
# B3.........Green............543-578 (nm)
# B4.........Red..............650-680 (nm)
# B5.........Red Edge 1.......698-713 (nm)
# B6.........Red Edge 2.......733-748 (nm)
# B7.........Red Edge 3.......773-793 (nm)
# B8.........NIR..............785-899 (nm)
# B8A........Red Edge 4.......855-875 (nm)
# B9.........Water Vapor......930-950 (nm)
# B10........Cirrus...........1360-1390 (nm)
# B11........SWIR 1...........1565-1655 (nm)
# B12........SWIR 2...........2100-2280 (nm)

# Create a dictionary of correspondence between original and new names
corresponding_name = {
    "ID_Unico": "ID_Unico",
    "SR_AV_B1": "Aerosol_Sentinel2A_resampled",
    "SR_AV_B2": "Blue_Sentinel2A_resampled",
    "SR_AV_B3": "Green_Sentinel2A_resampled",
    "SR_AV_B4": "Red_Sentinel2A_resampled",
    "SR_AV_B5": "Red_Edge_1_Sentinel2A_resampled",
    "SR_AV_B6": "Red_Edge_2_Sentinel2A_resampled",
    "SR_AV_B7": "Red_Edge_3_Sentinel2A_resampled",
    "SR_AV_B8": "NIR_Sentinel2A_resampled",
    "SR_AV_B8A": "Red_Edge_4_Sentinel2A_resampled",
    "SR_AV_B9": "Water_Vapor_Sentinel2A_resampled",
    "SR_AV_B10": "Cirrus_Sentinel2A_resampled",
    "SR_AV_B11": "SWIR_1_Sentinel2A_resampled",
    "SR_AV_B12": "SWIR_2_Sentinel2A_resampled"
}

# Rename the columns of the resampled_spectra dataframe
resampled_spectra.columns = [corresponding_name[col] for col in resampled_spectra.columns]

# Merge the dataframes
DB_raw_resampled_spectra = pd.merge(DB_raw, resampled_spectra, on="ID_Unico")

In [ ]:
print(DB_raw_resampled_spectra)

In [ ]:
# Saving Dataframes to CSV
DB_raw_resampled_spectra.to_csv("DB_validation_raw_resampled_spectra.csv", index=False)

# **Step 2° - Filter by equations**

## **Step 2.2 - Filter equation tendency**

The tendency equation is a rule that expresses the spectral signature of the soil. The spectral bands Blue, Green, Red, and NIR must necessarily follow the order of increasing values. This order is: Blue < Green < Red < NIR.

In [ ]:
import os
import pandas as pd

# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering")

# Reading the CSV file
DB_raw = pd.read_csv("DB_raw_resampled_spectra.csv", sep=',')

# Exibindo o DataFrame incial
print(DB_raw)

In [ ]:
import pandas as pd

# Creating the new DataFrame
results_equation_tendency = pd.DataFrame()

# Adding the 'ID_Unico' column to the new DataFrame
results_equation_tendency['ID_Unico'] = DB_raw['ID_Unico']

############################################################################
# Change the names of the spectral bands, Blue, Green, Red, and NIR, if
# the names of these respective bands in the columns of your dataset
# are different.

# Adding the 'outlier_tendency' column based on the condition
condition = (
    (DB_raw['Blue_SySI_Sentinel'] < DB_raw['Green_SySI_Sentinel']) &
    (DB_raw['Green_SySI_Sentinel'] < DB_raw['Red_SySI_Sentinel']) &
    (DB_raw['Red_SySI_Sentinel'] < DB_raw['NIR_SySI_Sentinel'])
)
############################################################################

results_equation_tendency['outlier_tendency'] = ~condition

# Displaying the new DataFrame
#print(results_equation_tendency)

# Adding the 'outlier_tendency' column to the original DataFrame using merge
DB_tendency = pd.merge(DB_raw, results_equation_tendency[['ID_Unico', 'outlier_tendency']], on='ID_Unico')

# Storing the 'outlier_tendency' column in a temporary variable
outlier_tendency_col = DB_tendency['outlier_tendency']

# Removing the 'outlier_tendency' column from the DataFrame
DB_tendency = DB_tendency.drop('outlier_tendency', axis=1)

# Inserting the 'outlier_tendency' column at the desired position (position 2)
DB_tendency.insert(2, 'outlier_tendency', outlier_tendency_col)

# DataFrame for True values in 'outlier_tendency'
outliers_samples_tendency = DB_tendency[DB_tendency['outlier_tendency'] == True]

# DataFrame for False values in 'outlier_tendency'
DB_filter_tendency = DB_tendency[DB_tendency['outlier_tendency'] == False]

# Displaying the final DataFrame
print(DB_filter_tendency)


In [ ]:
# Checking the number of samples in the initial dataframe,
# outliers tendency dataframe, and filtered dataframe

# Print the number of samples in DB_filter_lignin
print("Number of samples in DB_raw:", DB_raw.shape[0])

# Print the number of samples in outliers_samples_tendency
print("Number of samples in outliers_samples_tendency:", outliers_samples_tendency.shape[0])

# Print the number of samples in DB_filter_tendency
print("Number of samples in DB_filter_tendency:", DB_filter_tendency.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_spectra(data, plot_name: str = 'spectra_plot'):
    # Dicionário para mapear nomes de colunas para rótulos mais curtos
    label_map = {
        'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
        'Green_SySI_Sentinel': 'Green (543-578 nm)',
        'Red_SySI_Sentinel': 'Red (650-680 nm)',
        'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
        'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
        'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
        'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
        'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
        'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
        'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
    }

    # Especificar as colunas espectrais
    spectral_columns = list(label_map.keys())

    # Filtrar colunas que existem no DataFrame
    existing_columns = [col for col in spectral_columns if col in data.columns]
    # Obter rótulos curtos
    short_labels = [label_map[col] for col in existing_columns]

    plt.figure(figsize=(12, 7))

    # Iterando sobre cada linha nos dados
    for index, row in data.iterrows():
        # Plotar cada linha com base nas colunas espectrais
        plt.plot(short_labels, row[existing_columns], label=f'Outlier {index}')

    plt.xlabel('Spectral Band')
    plt.ylabel('Reflectance Factor')
    plt.title(f'Spectral Signatures of {plot_name}')

    # Rotacionar os labels do eixo x para evitar sobreposição
    plt.xticks(rotation=45)

    # Definir limites para o eixo y
    plt.ylim(0, 0.8)  # Ajuste conforme necessário

    plt.grid(False)

    # Salvar o gráfico com alta resolução
    output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/DB_Raw.png"
    dpi_value = 1200
    plt.savefig(output_path, dpi=dpi_value, bbox_inches='tight')

    plt.show()


In [ ]:
plot_spectra(DB_filter_tendency, 'DB Raw')

In [ ]:
# Save the DB_filter_step_2 DataFrame as a CSV file
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering")

DB_filter_tendency.to_csv("DB_filter_step_2_SySI_data.csv", index=False)
outliers_samples_tendency.to_csv("outliers_samples_tendency.csv", index=False)

# **Step 3° - Filter by PCA and distance Mahalanobis**

In [ ]:
# Packages required
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os

In [ ]:
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering")

# Importing soil spectra
DB_filter_step_2 = pd.read_csv("DB_filter_step_2_SySI_data.csv", low_memory=False)
DB_filter_step_2 = DB_filter_step_2.set_index('ID_Unico') #Assigning row names from ID column

print(DB_filter_step_2)

In [ ]:
### spectra selection ###

# Specify the index of the desired columns
col_index_min = 12  # Index of the initial column (adjust as needed)
col_index_max = 21 # Index of the final column (adjust as needed)

# Select columns based on index
data = DB_filter_step_2.iloc[:, col_index_min:col_index_max + 1]  # soil spectra (original)

print(data)

In [ ]:
### standardized data ###

data_selected = data.apply(pd.to_numeric, errors='coerce')
data_selected.fillna(data_selected.min(), inplace=True)

# Standardize the data
scaler = MinMaxScaler()
data_standardized = pd.DataFrame(scaler.fit_transform(data), index=data.index)

# Convert the standardized data back to a DataFrame
data_standardized_df = pd.DataFrame(data_standardized)

In [ ]:
print(data_standardized)

In [ ]:
#IDENTIY OUTLIERS FUNCTION
def identify_outliers(x, n_pc_comp=0):
    # Standardize the data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    # Handle NaN values (replace with mean of each column)
    x_scaled = np.nan_to_num(x_scaled, nan=np.nanmean(x_scaled, axis=0))

    # Perform PCA
    pca = PCA()
    pr_scores = pca.fit_transform(x_scaled)
    explained_variance = pca.explained_variance_ratio_
    cpr = np.cumsum(explained_variance)

    # Determine number of principal components
    if n_pc_comp == 0:
        n_pc_comp = np.min(np.where(cpr > 0.999)[0] + 1)
    elif 0 < n_pc_comp < 1:
        n_pc_comp = np.min(np.where(cpr > n_pc_comp)[0] + 1)

    # Calculate mean and covariance
    mean_pcaA = np.mean(pr_scores[:, :n_pc_comp], axis=0)
    cov_pcaA = np.cov(pr_scores[:, :n_pc_comp], rowvar=False)

    # Calculate Mahalanobis distance
    chiMat = np.zeros((pr_scores.shape[0], 3))
    chiMat[:, 0] = np.array([np.dot(np.dot((pr_scores[i, :n_pc_comp] - mean_pcaA), np.linalg.inv(cov_pcaA)), (pr_scores[i, :n_pc_comp] - mean_pcaA).T) for i in range(pr_scores.shape[0])])

    # Fit Chi-Squared distribution
    chiMat[:, 1] = chi2.cdf(chiMat[:, 0], df=n_pc_comp)

    # Determine critical value
    if n_pc_comp <= 10:
        pcrit = 1 - ((0.24 - 0.003 * n_pc_comp) / np.sqrt(pr_scores.shape[0]))
    else:
        pcrit = 1 - ((0.25 - 0.0018 * n_pc_comp) / np.sqrt(pr_scores.shape[0]))
    pcrit = 0.999
    #pcrit = 0.995

    # Identify outliers
    chiMat[:, 2] = np.where(chiMat[:, 1] >= pcrit, 0, 1)

    # Return the unique IDs of the outliers
    outliers_indices = np.where(chiMat[:, 2] == 0)[0]
    outliers_ids = x.index[outliers_indices]

    return outliers_ids


In [ ]:
# Calling the function and passing the original DataFrame
outliers_ids = identify_outliers(data, n_pc_comp=0.995)

# Exibir os IDs únicos dos outliers
print(outliers_ids)

In [ ]:
filtered_data_step_3 = data.drop(index=outliers_ids)
outliers_data_step_3 = data.loc[outliers_ids]

print(outliers_data_step_3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
import os

def plot_spectra(data, plot_name: str = 'spectra_plot', save_path=None, dpi: int = 1000):
    # Dicionário para mapear nomes de colunas para rótulos mais curtos
    label_map = {
        'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
        'Green_SySI_Sentinel': 'Green (543-578 nm)',
        'Red_SySI_Sentinel': 'Red (650-680 nm)',
        'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
        'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
        'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
        'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
        'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
        'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
        'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
    }
    # Especificar as colunas espectrais
    spectral_columns = list(label_map.keys())
    # Filtrar colunas que existem no DataFrame
    existing_columns = [col for col in spectral_columns if col in data.columns]
    # Obter rótulos curtos
    short_labels = [label_map[col] for col in existing_columns]
    plt.figure(figsize=(12, 6))

    # Iterando sobre cada linha nos dados
    for index, row in data.iterrows():
        # Plotar cada linha com base nas colunas espectrais
        plt.plot(short_labels, row[existing_columns], label=f'Outlier {index}')

    plt.xlabel('Spectral Band')
    plt.ylabel('Reflectance Factor')
    plt.title(f'Spectral Signatures of {plot_name}')

    # Rotacionar os labels do eixo x para evitar sobreposição
    plt.xticks(rotation=45)
    # Definir limites para o eixo y
    plt.ylim(0, 0.8)  # Ajuste conforme necessário
    plt.grid(False)
    plt.savefig(f'{plot_name}.png', dpi=dpi, bbox_inches='tight')
    plt.show()


def plot_3d_pca_with_outliers(data_standardized, outliers_ids, save_path=None, dpi: int = 1000):
    """
    Plots a 3D PCA plot highlighting outliers.

    Parameters:
    - data_standardized: The standardized data (samples x features).
    - outliers_ids: Indices of the outliers in the dataset.
    - save_path: (Optional) Path to save the plot. If None, the plot is not saved.
    """
    # Handle NaN values before applying PCA
    data_standardized = data_standardized.fillna(data_standardized.mean()) # Fill NaN with column means

    # Perform PCA
    pca = PCA(n_components=3)
    pr_scores = pd.DataFrame(pca.fit_transform(data_standardized), index=data_standardized.index)

    # Extracting the outliers and non-outliers
    outliers = pr_scores.loc[outliers_ids]
    non_outliers = pr_scores.drop(index=outliers_ids)

    # Convert outliers and non_outliers to numpy arrays for plotting
    outliers_array = outliers.to_numpy()
    non_outliers_array = non_outliers.to_numpy()

    # Create 3D plot
    fig = plt.figure(figsize=(14, 16))
    ax = fig.add_subplot(111, projection='3d')

    # Plot outliers
    ax.scatter(outliers_array[:, 0], outliers_array[:, 1], outliers_array[:, 2], c='red', marker='x', s=50, label="Outliers")

    # Plot non-outliers
    ax.scatter(non_outliers_array[:, 0], non_outliers_array[:, 1], non_outliers_array[:, 2], c='gray', marker='o', alpha=0.6, label="Non-outliers")

    # Label axes with explained variance ratios
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0] * 100:.2f}%)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1] * 100:.2f}%)')
    ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2] * 100:.2f}%)')

    # Add legend
    ax.legend()

    # Set the plot title
    plt.title('3D PCA plot with Outliers')

    # Save the figure if a path is provided
    if save_path:
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight')

    # Show plot
    plt.show()

In [ ]:
# Plots Visualization

# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_3_SySI_data")

plot_spectra(filtered_data_step_3, 'filtered_data_step_3')
plot_spectra(outliers_data_step_3, 'outliers')
plot_spectra(data, 'all_data')
plot_3d_pca_with_outliers(data, outliers_ids, save_path='3d_pca_outliers')

In [ ]:
outliers_data_step_3 = outliers_data_step_3.reset_index()

IDs_to_filter = outliers_data_step_3['ID_Unico'].tolist()

DB_filter_step_3 = DB_filter_step_2.reset_index()

# Add a new column "outliers_step_3" filled with False
DB_filter_step_3.insert(1, "outliers_step_3", False)

# Update the values to True where ID_Unico is in the list IDs_to_filter
DB_filter_step_3.loc[DB_filter_step_3['ID_Unico'].isin(IDs_to_filter), 'outliers_step_3'] = True

# Create a new DataFrame with only the rows where outliers_step_3 is True
outliers_samples_step_3 = DB_filter_step_3[DB_filter_step_3['outliers_step_3'] == True].copy()

# Remove the samples where outliers_step_3 is True
DB_filter_step_3 = DB_filter_step_3[DB_filter_step_3['outliers_step_3'] == False].copy()

print(outliers_samples_step_3)

In [ ]:
# Checking the number of samples in the initial dataframe,
# outliers_samples_step_3 dataframe, and filtered dataframe

# Print the number of samples in DB_filter_step_2
print("Number of samples in DB_filter_step_2:", DB_filter_step_2.shape[0])

# Print the number of samples in outliers_step_3
print("Number of samples in outliers_samples_step_3:", outliers_samples_step_3.shape[0])

# Print the number of samples in DB_filter_step_3
print("Number of samples in DB_filter_step_3:", DB_filter_step_3.shape[0])

In [ ]:
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_3_SySI_data")

# Save the outliers_samples_3 DataFrame as a CSV file
outliers_samples_step_3.to_csv("outliers_samples_step_3.csv", index=False)

# Save the DB_filter_3 DataFrame as a CSV file
DB_filter_step_3.to_csv("DB_validation_filter_step_3.csv", index=False)

## **Visual analysis of the outiliers determined by Step 3°**

This sub-step is not mandatory for completing the entire protocol, but we strongly recommend that the spectral signatures of samples identified as outliers be visually analyzed. This analysis allows the pedometrist to understand the nature of the outlier spectrum and why the data might be inconsistent or incorrect. Examples include:

- Miscalibration of the Vis-NIR-SWIR sensor, resulting in noise in the spectral signature.
- Contamination of the soil sample with other materials.
- Improper preparation of the soil sample.
- Collection of the spectral data from a target other than the soil sample.

By understanding the nature of the error, the pedometrist can intervene at the source, preventing similar errors from occurring in the future.
However, it is important to note that the spectral data may be correct. The combination of PCA and Mahalanobis distance may flag an outlier that actually corresponds to a soil with distinct spectral behavior, which is not well-represented in the dataset. Therefore, if you identify a sample whose spectral behavior matches all the correct spectral characteristics of a soil in the "outliers_samples_step_3" dataset, we recommend including this sample's data in the "DB_filter_step_3" dataset.

The following interactive plots of the hyperspectral data from soil samples will be generated for visual analysis.

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
print(outliers_samples_step_3)

In [ ]:
# Define the indices of the columns you want to select
col_index_min = 109  # Index of the first spectral column
col_index_max = 2189  # Index of the last spectral column

# Select the ID column by name and the spectral columns by indices
selected_columns = [outliers_samples_step_3.columns.get_loc('ID_Unico')] + list(range(col_index_min, col_index_max + 1))

# Create a new DataFrame with the desired columns
DB_graphic = outliers_samples_step_3.iloc[:, selected_columns].copy()

print(DB_graphic)

In [ ]:
# Split the dataframe into smaller dataframes based on rows, in this case 50 rows
dfs = np.array_split(DB_graphic, np.ceil(len(DB_graphic) / 100))

# Create the line chart using Plotly Express for each smaller dataframe
for i, df in enumerate(dfs):
    # Reshape the dataframe
    df_melt = df.melt(id_vars=df.columns[0], value_vars=df.columns[1:])

    fig = px.line(df_melt, x='variable', y='value', color=df.columns[0],
                  title=f'Original Spectra {i+1}', labels={'variable': 'Wavelength (nm)', 'value': 'Reflectance Factor'})

    # Customize the chart layout
    fig.update_layout(
        xaxis_title='Wavelength (nm)',
        yaxis_title='Reflectance Factor',
    )

    # Build the HTML file name based on the chart
    html_file_name = os.path.join('G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/Step_3', f'Original_Spectra_Plotly_{i+1}.html')

    # Save the interactive chart to an HTML file
    fig.write_html(html_file_name)

    # Display the interactive chart
    fig.show()

# **Step 4° - Filter by Soil Line**

In [ ]:
# Packages required
import os
import statsmodels.api as sm
import numpy as np
import pandas as pd
from plotnine import *
from plotnine import ggsave
import scipy.stats as stats
from plotnine import ggplot

In [ ]:
### Soil line on convolved spectra ###

# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering\Step_3_SySI_data")

# Read the CSV file
DB_filter_step_3 = pd.read_csv("DB_validation_filter_step_3.csv")

# Assigning to a variable for clarity
DB = DB_filter_step_3

print(DB_filter_step_3)

In [ ]:
### Cutting line determination ###

# Specify the X and Y columns
column_x_conv = DB['Red_SySI_Sentinel']
column_y_conv = DB['NIR_SySI_Sentinel']

# Create a linear regression model
X = sm.add_constant(column_x_conv)  # Add the constant for the intercept term
model = sm.OLS(column_y_conv, X).fit()

# Calculate predicted values
predictions = model.predict(X)

# Calculate the standard deviation of residuals
residuals = model.resid
std_residuals = np.std(residuals)

# Confidence level (e.g., 98%)
confidence_level = 0.99

# Quantile of the t distribution
quantile_t = stats.t.ppf((1 + confidence_level) / 2, df=len(residuals) - 2)

# Calculate upper and lower confidence limits
upper_limit_NIR_resam = predictions + quantile_t * (std_residuals * 1.25)
lower_limit_NIR_resam = predictions - quantile_t * (std_residuals * 1.25)

# Regression coefficients
intercept, coef_x = model.params

# Standard error of coefficients
std_error = model.bse

# T-value and p-value of coefficients
t_value = model.tvalues
p_value = model.pvalues

# Coefficient of determination (R²)
r_squared_convolved = model.rsquared

# Fit a linear regression model for the upper limit
model_upper_limit = sm.OLS(upper_limit_NIR_resam, X).fit()

# Fit a linear regression model for the lower limit
model_lower_limit = sm.OLS(lower_limit_NIR_resam, X).fit()

# Coefficients of intercept and slope for the upper limit
coef_intercept_upper, coef_slope_upper = model_upper_limit.params

# Coefficients of intercept and slope for the lower limit
coef_intercept_lower, coef_slope_lower = model_lower_limit.params

# Display coefficients
print("Coefficients for Upper Limit:")
print("Intercept (β0):", round(coef_intercept_upper, 4))
print("Slope (β1):", round(coef_slope_upper, 4))

print("\nCoefficients for Lower Limit:")
print("Intercept (β0):", round(coef_intercept_lower, 4))
print("Slope (β1):", round(coef_slope_lower, 4))

In [ ]:
### plot the Soil line by convolved data ###

# Desired width and height of the plot
plot_width = 9
plot_height = 6

soil_line_plot = (
    ggplot(DB, aes(x='Red_SySI_Sentinel', y='NIR_SySI_Sentinel')) +
    theme_classic() +
    geom_point(shape='o', color= 'black', fill= "#E49247", alpha=1, size=1.5, stroke=0.2) + # Data points with black contour +  # Data points
    geom_smooth(method='lm', se=False, color='#000000', size=1.2) +  # Trend line
    geom_line(aes(y=upper_limit_NIR_resam), color='#E41A1C', size=1) +  # Upper confidence interval line
    geom_line(aes(y=lower_limit_NIR_resam), color='#E41A1C', size=1) +  # Lower confidence interval line

    annotate('text', x=0.11, y=0.58, label="β upper limit: {:.3f}".format(round(coef_intercept_upper, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.54, label="β lower limit: {:.3f}".format(round(coef_intercept_lower, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.50, label="R² model: {:.3f}".format(round(r_squared_convolved, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.46, label="α: {:.3f}".format(round(coef_x, 3)),
             family="Arial", size=10)+

    labs(x="SySI Red band (650-680 nm)", y="SySI NIR band (785-899 nm)") +
    expand_limits(x=0, y=0) +  # Force the plot to show the origin (0,0)
    scale_y_continuous(limits=[0, 0.6]) +
    scale_x_continuous(limits=[0, 0.6]) +
    theme(
        axis_title_x=element_text(margin={'t': 10}, family="Arial", size=12, face="bold"),
        axis_title_y=element_text(margin={'r': 10}, family="Arial", size=12, face="bold"),
        axis_text_y=element_text(family="Arial", size=10),
        axis_text_x=element_text(family="Arial", size=10)
    )
)


# Display the plot
print(soil_line_plot)

dpi_value = 1000

# Setting the working directory
output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_4_SySI_data/soil_line_plot.png"
soil_line_plot.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)


In [ ]:
## After Soil Line analysis, filtering the samples in the database ###

# Create a subset of samples that are ABOVE the upper_limit_NIR line
samples_upper_limit = DB[DB['NIR_SySI_Sentinel'] > upper_limit_NIR_resam]

# Create a subset of samples that are BELOW the lower_limit_NIR line
samples_lower_limit = DB[DB['NIR_SySI_Sentinel'] < lower_limit_NIR_resam]

# Concatenate the dataframes samples_upper_limit and samples_lower_limit
outliers_samples = pd.concat([samples_upper_limit, samples_lower_limit])

# Merge the DataFrames based on the ID_Unico column and keep only the columns from the DB DataFrame
outliers_samples_step_4 = pd.merge(outliers_samples[['ID_Unico']], DB, on='ID_Unico', how='inner')

IDs_to_filter = outliers_samples['ID_Unico'].tolist()

DB_filter_step_4 = DB_filter_step_3

# Add a new column "outliers_step_4" filled with False
DB_filter_step_4.insert(1, "outliers_step_4", False)

# Update the values to True where ID_Unico is in the list IDs_to_filter
DB_filter_step_4.loc[DB_filter_step_4['ID_Unico'].isin(IDs_to_filter), 'outliers_step_4'] = True

# Create a new DataFrame with only the rows where outliers_step_4 is True
outliers_samples_step_4 = DB_filter_step_4[DB_filter_step_4['outliers_step_4'] == True].copy()

# Delete the samples where outliers_step_3 is True
DB_filter_step_4 = DB_filter_step_4[DB_filter_step_4['outliers_step_4'] == False].copy()

print(outliers_samples_step_4)

In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_4, and DB_filter_step_4 dataframe

# Print the number of samples in DB_filter_step_3
print("Number of samples in DB_filter_step_3:", DB_filter_step_3.shape[0])

# Print the number of samples in outliers_samples_step_4
print("Number of samples in outliers_samples_step_4:", outliers_samples_step_4.shape[0])

# Print the number of samples in DB_filter_step_4
print("Number of samples in DB_filter_step_4:", DB_filter_step_4.shape[0])

In [ ]:
# Save the DB_filter_step_4 and outliers_samples_step_4 DataFrame as a CSV file
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_4_SySI_data")

outliers_samples_step_4.to_csv("outliers_samples_step_4_SySI_data.csv", index=False)

DB_filter_step_4.to_csv("DB_filter_step_4_SySI_data.csv", index=False)

# **Step 5° - CIFOD Filter**

## URSSA module 1: Importing and pre-processing data

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import pairwise_distances
from scipy.interpolate import interp1d
from plotnine import ggplot

In [ ]:
# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_4_SySI_data")

# Importing data (soil attributes + spectra)
Data_all = pd.read_csv("DB_filter_step_4_SySI_data.csv")

In [ ]:
print(Data_all)

In [ ]:
# Copia o DataFrame Data_all para Data
Data = Data_all

# Especifica o intervalo das colunas espectrais
col_index_min = 15  # Índice da primeira coluna espectral
col_index_max = 24  # Índice da última coluna espectral

# Seleciona as colunas espectrais
Spectra_original = Data.iloc[:, col_index_min:col_index_max+1]  # Espectros do solo (original)

# Seleciona a coluna ID_Unico
ID_Unico = Data['ID_Unico']

# Cria o DataFrame dataset_model contendo as colunas espectrais e a coluna ID_Unico
dataset_model = Spectra_original.copy()

# Insere a coluna ID_Unico como a primeira coluna
dataset_model.insert(0, 'ID_Unico', ID_Unico)

# Plotting of the first ten original spectra
plt.figure()
plt.plot(Spectra_original.columns, Spectra_original.iloc[:10].T.values)  # Use column names as is (without int conversion)
plt.title('Original Spectra')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance Factor')
plt.xticks(rotation=90)  # Rotate x labels if needed
plt.show()

In [ ]:
print(dataset_model)

## URSSA module 2: Unsupervised spectral clustering of samples


The script can be executed in two ways: hosted on a personal machine or through the Google API.

### ***API Google***

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages('randomUniformForest')
utils.install_packages('magrittr')
utils.install_packages('dplyr')
utils.install_packages('tidyverse')

In [ ]:
### MODULE 2: Unsupervised spectral clustering of samples ####

## 1) Unsupervised randomUniformForest classification,
## 2) Calculating spectral proximity from URF,
## 3) Multidimensional Scaling (MDS),
## 4) Clustering samples (K-means)


import rpy2.robjects as robjects
from rpy2.robjects.packages import importr


# Import the R packages
randomUniformForest = importr('randomUniformForest')
magrittr = importr('magrittr')
dplyr = importr('dplyr')
tidyverse = importr('tidyverse')


dataset_model.to_csv("dataset_model.csv", index=True)

# Define the R script
r_script = """

setwd("/content")
dataset_model <- read.csv("dataset_model.csv", header = TRUE, sep = ",", dec = ".", na.strings = "NA", check.names = FALSE)
dataset_model <- dataset_model %>% remove_rownames %>% column_to_rownames(var="ID_Unico")

wavelength_min <- "350"
wavelength_max <- "2500"

results <- unsupervised.randomUniformForest(object = dataset_model %>% select(wavelength_min:wavelength_max), # select spectra
                                              baseModel = "proximity", # c("proximity", "proximityThenDistance", "importanceThenDistance")
                                              endModel = "MDSkMeans", # c("MDSkMeans", "MDShClust", "MDS", "SpectralkMeans")
                                              endModelMetric = NULL,
                                              samplingMethod = "with bootstrap", # c("uniform univariate sampling","uniform multivariate sampling", "with bootstrap")
                                              MDSmetric = "metricMDS", # c("metricMDS", "nonMetricMDS")
                                              proximityMatrix = NULL,
                                              sparseProximities = FALSE,
                                              outliersFilter = FALSE,
                                              Xtest = NULL,
                                              predObject = NULL,
                                              metricDimension = 2,
                                              coordinates = c(1,2),
                                              bootstrapReplicates = 100,
                                              clusters = NULL,
                                              maxIters = NULL,
                                              importanceObject = NULL,
                                              maxInteractions = 2,
                                              reduceClusters = FALSE,
                                              maxClusters = 10,
                                              mapAndReduce = FALSE,
                                              OOB = FALSE,
                                              subset = NULL,
                                              seed = 2014,
                                              uthreads = "auto")

results_final_R <- data.frame("ID_Unico" = rownames(dataset_model),
                              "cluster" = results$unsupervisedModel$cluster,
                              dataset_model)

write.table(results_final_R, "results_final_R.csv", sep = ",", dec = ".",row.names = FALSE)

"""

# Execute the R script
robjects.r(r_script)


### ***Personal machine - local execution environment:***
  Attention, you must indicate the directory path where R is installed. The "randomUniformForest" library requires version 4.3.2. And within the script in R language it is necessary to indicate the path to the directory where the libraries are installed.

In [ ]:
### MODULE 2: Unsupervised spectral clustering of samples ####

## 1) Unsupervised randomUniformForest classification,
## 2) Calculating spectral proximity from URF,
## 3) Multidimensional Scaling (MDS),
## 4) Clustering samples (K-means)

############################################################################################
#  enter the directory of the R language installed on the machine
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.3.2'
############################################################################################

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Saving Dataframes to CSV

os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data")
dataset_model.to_csv("dataset_model.csv", index=True)

# Define the R script
r_script = """

############################################################################################
## R: Directory path to the required hsdar package libraries##                             #
#                                                                                          #
 .libPaths("G:/OneDrive/Documentos/Doutorado/protocol_database/libraires")                 #
#                                                                                          #
############################################################################################

# Load the doParallel library
library(doParallel)

# Configure parallel cluster (can use desired number of cores)
cl <- makeCluster(16)
registerDoParallel(cl)

library(dplyr)
library(tidyverse)
library(randomUniformForest)

############################################################################################

setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data")

############################################################################################

dataset_model <- read.csv("dataset_model.csv", header = TRUE, sep = ",", dec = ".", na.strings = "NA", check.names = FALSE)
dataset_model <- dataset_model %>% remove_rownames %>% column_to_rownames(var="ID_Unico")

wavelength_min <- "Blue_SySI_Sentinel"
wavelength_max <- "SWIR_2_10m_SySI_Sentinel"

results <- unsupervised.randomUniformForest(object = dataset_model %>% select(wavelength_min:wavelength_max), # select spectra
                                              baseModel = "proximity", # c("proximity", "proximityThenDistance", "importanceThenDistance")
                                              endModel = "MDSkMeans", # c("MDSkMeans", "MDShClust", "MDS", "SpectralkMeans")
                                              endModelMetric = NULL,
                                              samplingMethod = "uniform multivariate sampling", # c("uniform univariate sampling","uniform multivariate sampling", "with bootstrap")
                                              MDSmetric = "metricMDS", # c("metricMDS", "nonMetricMDS")
                                              proximityMatrix = NULL,
                                              sparseProximities = FALSE,
                                              outliersFilter = FALSE,
                                              Xtest = NULL,
                                              predObject = NULL,
                                              metricDimension = 2,
                                              coordinates = c(1,2),
                                              bootstrapReplicates = 100,
                                              clusters = NULL,
                                              maxIters = 300,
                                              importanceObject = NULL,
                                              maxInteractions = 5,
                                              reduceClusters = FALSE,
                                              maxClusters = 7,
                                              mapAndReduce = FALSE,
                                              OOB = FALSE,
                                              subset = NULL,
                                              seed = 2014,
                                              uthreads = "auto")

results_final_R <- data.frame("ID_Unico" = rownames(dataset_model),
                              "cluster" = results$unsupervisedModel$cluster,
                              dataset_model)

write.table(results_final_R, "results_final_R.csv", sep = ",", dec = ".",row.names = FALSE)

"""

# Execute the R script
robjects.r(r_script)

In [ ]:
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data")
results_final = pd.read_csv("results_final_R.csv")

print(results_final)

In [ ]:
### manipulating results in dataframes ###

clusters = results_final

# Selecting only the 'ID_Unico' and 'cluster' columns from the clusters DataFrame
clusters = clusters[['ID_Unico', 'cluster']]

# Performing a merge between 'clusters' and 'Data_all' using 'ID_Unico' as the key
# This will join both DataFrames, keeping only the rows with matching 'ID_Unico'
Soil_data_clustered = pd.merge(clusters, Data_all, on='ID_Unico', how='inner')

# Saving the merged DataFrame 'Soil_data_clustered' to a CSV file
# The file is saved with a comma as the separator and a dot for decimals
Soil_data_clustered.to_csv('Soil_data_clustered.csv', sep=',', decimal='.', index=False)

##Identification of soil attribute outliers by Isolation Forest in each cluster

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from numpy import where
from mpl_toolkits.mplot3d import Axes3D
from plotnine import *

In [ ]:
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data")
Soil_data_clustered = pd.read_csv("Soil_data_clustered.csv")

print(Soil_data_clustered)

In [ ]:

# Create an empty DataFrame called results
results = pd.DataFrame()

# Loop through clusters from 1 to 10
for i in range(1, 11):
    # Filter the DataFrame to select only the records where the "cluster" column is equal to i
    data_estratificado = Soil_data_clustered[Soil_data_clustered['cluster'] == i]

    # Find the columns starting with "Blue_SySI_Sentinel" and ending with "SWIR_2_10m_SySI_Sentinel"
    colunas_selecionadas = data_estratificado.loc[:, 'Blue_SySI_Sentinel':'SWIR_2_10m_SySI_Sentinel']

    # Calculate the mean and standard deviation of the selected columns
    medias = colunas_selecionadas.mean(numeric_only=True)
    desvio_padrao = colunas_selecionadas.std(numeric_only=True)

    # Create a temporary DataFrame with the results for the current cluster
    results_cluster = pd.DataFrame({
        'cluster': [i] * len(medias),
        'wavelength': medias.index,  # Keep the column names as strings
        'average_reflectance': medias.values,
        'sd': desvio_padrao.values
    })

    # Add the results of the current cluster to the main DataFrame
    results = pd.concat([results, results_cluster], ignore_index=True)

# Delete rows where 'average_reflectance' is NaN (null)
results = results.dropna(subset=['average_reflectance'])

# Dicionário para mapear nomes de colunas para rótulos mais curtos
label_map = {
    'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
    'Green_SySI_Sentinel': 'Green (543-578 nm)',
    'Red_SySI_Sentinel': 'Red (650-680 nm)',
    'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
    'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
    'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
    'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
    'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
    'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
    'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
}

# Atualiza os rótulos das colunas no DataFrame de resultados
results['wavelength'] = results['wavelength'].map(label_map).fillna(results['wavelength'])

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3","#FF760A",
           "#FFD700","#0AB2D1", "#F781BF", "#999999", "#66C2A5", "#8C564B",
           "#1F78B4", "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Desired width and height of the plot
plot_width = 14
plot_height = 7

spectral_behavior = (ggplot(results, aes(x='wavelength',
                                         y='average_reflectance',
                                         ymin='average_reflectance - sd',
                                         ymax='average_reflectance + sd',
                                         group='cluster')) +

  geom_line(aes(color='factor(cluster)'), size=0.85) +
  geom_ribbon(aes(fill='factor(cluster)'), alpha=0.2) +
  scale_y_continuous(limits=[0, 0.7]) +
  scale_color_manual(values=palette) +  # Manually set the colors
  scale_fill_manual(values=palette) +  # Manually set the fill colors
  labs(title='Spectral behavior of clusters',
       x='Wavelength (nm)',
       y='Reflectance factor',
       color='Cluster',   # Name the color legend
       fill='Cluster') +  # Name the fill legend
  theme_minimal() +
  theme(figure_size=(plot_width, plot_height),
        plot_title=element_text(size=15, face="bold"),
        axis_title_x=element_text(size=12, face="bold"),
        axis_title_y=element_text(size=12, face="bold"),
        axis_text_x=element_text(size=10, angle=45, hjust=1),  # Rotate x-axis labels
        legend_text=element_text(size=10),
        legend_title=element_text(size=12, face="bold"),
        plot_background=element_rect(fill="white")))

print(spectral_behavior)

dpi_value = 1200

# Setting the working directory
output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/SySI_spectral_behavior_of_clusters.png"
spectral_behavior.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

In [ ]:
# Data preparation
Data = Soil_data_clustered.dropna(subset=['C_gkg'])
Data = Data.set_index('ID_Unico')

# Select the cluster column and the variable of interest
Observation = Data[["cluster", "C_gkg"]]

# Filter null data in the "Clay_gkg" column
Observation = Observation[Observation['C_gkg'].notnull()]

# Specify the range of spectral columns
col_index_min = 15  # Index of the first spectral column
col_index_max = 24 # Index of the last spectral column

# Select the spectral columns
Spectra_original = Data.iloc[:, col_index_min:col_index_max+1]

# Merge DataFrames to have all the information in one place
Observation_final = pd.merge(Observation, Spectra_original, left_index=True, right_index=True, how='left')

print(Observation_final)

In [ ]:
print(Observation_final)

In [ ]:
############ Isolation Forest ############

#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

# Carregar o dataset
data = Observation_final

# Remover a coluna 'cluster' para manter apenas as features espectrais
features = data.drop(columns=['cluster'])

# Inicializar dicionários para armazenar os resultados
anomalies_dict = {}
plot_data_dict = {}

# Iterar sobre cada cluster único
for idx, cluster_label in enumerate(sorted(data['cluster'].unique())):

    # Filtrar dados para o cluster atual
    cluster_data = data[data['cluster'] == cluster_label]

    # Remover linhas com valores ausentes e manter apenas as features espectrais
    data_filtered = cluster_data[features.columns].dropna()

    # Garantir que os nomes das colunas sejam strings
    data_filtered.columns = data_filtered.columns.astype(str)

    # Normalizar os dados usando StandardScaler
    scaler = StandardScaler()
    data_filtered_scaled = scaler.fit_transform(data_filtered)

    # Aplicar Isolation Forest para detectar anomalias
    iforest = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
    predictions = iforest.fit_predict(data_filtered_scaled)

    # Obter o score de anomalia
    anomaly_scores = iforest.decision_function(data_filtered_scaled)
    data_filtered['anomaly_score'] = anomaly_scores

    # Identificar os índices dos outliers (anomalias)
    anom_index = where(predictions == -1)
    anomalies = data_filtered.iloc[anom_index]
    anomalies['cluster'] = cluster_label  # Adicionar a coluna do cluster

    # Armazenar as anomalias no dicionário
    anomalies_dict[cluster_label] = anomalies

    # Aplicar PCA para reduzir as features espectrais para 2 componentes principais
    spectral_features = features.columns[1:]  # Excluir 'Clay_gkg'
    pca = PCA(n_components=2)
    data_pca = pca.fit_transform(data_filtered[spectral_features])

    # Criar um DataFrame com PCs e o atributo 'Clay_gkg'
    data_3d = pd.DataFrame(data_pca, columns=['PC1', 'PC2'])
    data_3d['C_gkg'] = data_filtered['C_gkg'].values

    # Separar os dados entre outliers e não-outliers
    outliers_3d = data_3d.iloc[anom_index]
    non_outliers_3d = data_3d.drop(index=anom_index[0])

    # Armazenar os dados para os plots no dicionário
    plot_data_dict[cluster_label] = {
        'data_3d': data_3d,
        'outliers_3d': outliers_3d,
        'non_outliers_3d': non_outliers_3d,
        'pca': pca
    }


In [ ]:
### 3D plot Visualization ###

# Define the color palette for the clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3",
           "#FF760A","#FFD700","#0AB2D1"]

# Output directory for the plots
output_dir = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/Organic_Carbon"

# Iterate over each cluster and generate plots
for idx, (cluster_label, results) in enumerate(plot_data_dict.items()):
    data_3d = results['data_3d']
    outliers_3d = results['outliers_3d']
    non_outliers_3d = results['non_outliers_3d']
    pca = results['pca']

    # Create 3D plot
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111, projection='3d')

    # Plot outliers
    ax.scatter(outliers_3d['PC1'], outliers_3d['PC2'], outliers_3d['C_gkg'],
               c='red', marker='x', s=110, label="Outliers")

    # Plot non-outliers with cluster-specific color
    cluster_color = palette[idx % len(palette)]
    ax.scatter(non_outliers_3d['PC1'], non_outliers_3d['PC2'], non_outliers_3d['C_gkg'],
               c=cluster_color, marker='o', alpha=1, edgecolors='black', s=60, label="Non-outliers")


    # Label axes with explained variance of PCs and Clay attribute, and increase font size
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0] * 100:.2f}%)', fontweight='bold', fontsize=35, labelpad=37)
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1] * 100:.2f}%)', fontweight='bold', fontsize=35, labelpad=45)
    ax.set_zlabel('C gkg', fontweight='bold', fontsize=35, labelpad=54)

    # Increase the font size of the tick labels
    ax.tick_params(axis='both', which='major', labelsize=33, pad=9)  # Adjusts x and y axis
    ax.tick_params(axis='z', which='major', labelsize=33, pad=24)     # Adjusts z axis

    # Set z-axis limits from 0 to 900
    ax.set_zlim(0, 250)

    # Adjust layout to prevent label cutoff
    plt.tight_layout()

    # Add legend and title
    ax.legend(fontsize=12)
    plt.title(f'3D PCA plot with Outliers for Cluster {cluster_label} (Organic Carbon gkg vs. PC1 & PC2)', fontweight='bold', fontsize=20, y=1.05)

    # Save the plot as an image with specified DPI
    output_path = f"{output_dir}cluster_{cluster_label}_outlier_detection.png"
    plt.savefig(output_path, dpi=1000)

    # Display the plot
    plt.show()


In [ ]:
# Concatenate all DataFrames in the dictionary into a single DataFrame
anomalies_combined = pd.concat(anomalies_dict.values())

# Reset the index of the combined DataFrame
anomalies_combined = anomalies_combined.reset_index()

print(anomalies_combined)

In [ ]:

# Generate a list of all sample IDs in anomalies_combined
IDs_to_filter = anomalies_combined['ID_Unico'].tolist()

# Print the list of sample IDs
print(IDs_to_filter)

# Summary of the number of samples per cluster
summary = anomalies_combined.groupby('cluster').size()

# Print the summary by cluster
print(summary)

In [ ]:
DB_filter_step_5 = Data.reset_index()

# Add a new column "outliers_step_5_" filled with False
DB_filter_step_5.insert(1, "outliers_step_5_C", False)

# Update the values to True where ID_Unico is in the list IDs_to_filter
DB_filter_step_5.loc[DB_filter_step_5['ID_Unico'].isin(IDs_to_filter), 'outliers_step_5_C'] = True

# Create a new DataFrame with only the rows where outliers_step_5_ is True
outliers_samples_step_5 = DB_filter_step_5[DB_filter_step_5['outliers_step_5_C'] == True].copy()

# Delete the samples where outliers_step_5_ is True
DB_filter_step_5 = DB_filter_step_5[DB_filter_step_5['outliers_step_5_C'] == False].copy()

print(outliers_samples_step_5)

In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_5, and DB_filter_step_5 dataframe

# Print the number of samples in Observation_final
print("Number of samples in Observation_final:", Observation_final.shape[0])

# Print the number of samples in outliers_samples_step_5_
print("Number of samples in outliers_samples_step_5_C:", outliers_samples_step_5.shape[0])

# Print the number of samples in DB_filter_step_5_
print("Number of samples in DB_filter_step_5_C:", DB_filter_step_5.shape[0])

In [ ]:
# Save the DB_filter_step_5 and outliers_samples_step_5 DataFrame as a CSV file

# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/Organic_Carbon")

# Save the outliers_samples_3 DataFrame as a CSV file
outliers_samples_step_5.to_csv("outliers_samples_step_5_C_3.csv", index=False)

# Save the DB_filter_3 DataFrame as a CSV file
DB_filter_step_5.to_csv("DB_filter_step_5_C_3.csv", index=False)

In [ ]:
from plotnine import *

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF760A","#FFD700",
           "#0AB2D1", "#F781BF", "#999999", "#66C2A5", "#8C564B","#1F78B4",
           "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Set the desired plot size
plot_width = 10
plot_height = 9

# Create the plot using plotnine
box_plot_1 = (ggplot(Observation_final, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 250]) +
            labs(title='Box Plot of Organic Carbon gkg for Each Cluster before filter',
                 x='Cluster',
                 y='C_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Display the plot
print(box_plot_1)

# Create the plot using plotnine
box_plot_2 = (ggplot(DB_filter_step_5, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 250]) +
            labs(title='Box Plot of Organic Carbon gkg for Each Cluster after filter',
                 x='Cluster',
                 y='C_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Display the plot
print(box_plot_2)

dpi_value = 1000

# Setting the working directory
output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/Organic_Carbon/box_plot_1.png"
box_plot_1.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/Organic_Carbon/box_plot_2.png"
box_plot_2.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from plotnine import ggplot, aes, geom_boxplot, scale_fill_manual, scale_y_continuous, labs, theme_minimal, theme, element_text, element_rect

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00",
           "#FFFF33", "#A65628", "#F781BF", "#999999", "#66C2A5", "#8C564B",
           "#1F78B4", "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Set the desired plot size
plot_width = 10
plot_height = 9

# Create the plot using plotnine for before the filter
box_plot_1 = (ggplot(Observation_final, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 200]) +
            labs(title='Box Plot of C gkg for Each Cluster before filter',
                 x='Cluster',
                 y='C_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Create the plot using plotnine for after the filter
box_plot_2 = (ggplot(DB_filter_step_5_C, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 200]) +
            labs(title='Box Plot of C gkg for Each Cluster after filter',
                 x='Cluster',
                 y='C_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Save the plots with high resolution (1000 dpi)
box_plot_1.save("box_plot_1.png", dpi=1000)
box_plot_2.save("box_plot_2.png", dpi=1000)

# Load the saved images
img1 = mpimg.imread("box_plot_1.png")
img2 = mpimg.imread("box_plot_2.png")

# Set up the side-by-side display
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(plot_width*2, plot_height))

# Display the images side by side
axes[0].imshow(img1)
axes[0].axis('off')  # Remove the axes
axes[0].set_title('Box Plot of C gkg for Each Cluster before filter')

axes[1].imshow(img2)
axes[1].axis('off')  # Remove the axes
axes[1].set_title('Box Plot of C gkg for Each Cluster after filter')

# Save the combined figure with high resolution (1000 dpi)
plt.tight_layout()
plt.savefig('combined_box_plots.png', dpi=1000)
plt.show()


# **Step 6° - ABOPA Filter**

## Importing and manipulating data

In [ ]:
# Packages required
import os
import pandas as pd

In [ ]:
# Initial Setting up

# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_5_SySI_data/Organic_Carbon")

# Read the CSV file
DB_filter_step_5 = pd.read_csv("DB_filter_step_5_C_3.csv")

# Assigning to a variable for clarity
DB = DB_filter_step_5

In [ ]:
print(DB)

In [ ]:
# Selecting specific columns where the spectrum begins and ends
DB_spectra = DB.iloc[:, 17:27]

print(DB_spectra)

In [ ]:

# Creating a new dataframe with specific columns and removing rows with NA values

DB1 = pd.concat([DB.iloc[:, [0, 9]], # select the column number where the dependent variable is

                 DB_spectra.iloc[:, list(range(0, 10, 1))]], axis=1)

DB1 = DB1.dropna()

# Printing the dataframe structure
print(DB1.info())

print(DB1)

# Selecting specific rows for training
DB_train = DB1.iloc[:, 2:12]

# Selecting a specific column
atr_content = DB1['C_gkg']

In [ ]:
print(DB_train)

## Prediction via Cubist algorithm

### ***Personal machine - local execution environment:***
  Attention, you must indicate the directory path where R is installed. The "Caret" library requires version 4.3.2. And within the script in R language it is necessary to indicate the path to the directory where the libraries are installed.

In [ ]:
### Cubist Algorithm with Cross Validation ###
# Imports
import os
# Se estiver executando o script na máquina pessoal, insira o diretório da linguagem R instalada na máquina
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.3.2'
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")

# Saving Dataframes to CSV
DB1.to_csv("DB1.csv", index=False)
DB_train.to_csv("DB_train.csv", index=False)
atr_content.to_csv("atr_content.csv", index=False)


# R Script
r_script = """

############################################################################################
## R: Directory path to the required package libraries##                                   #
#                                                                                          #
 .libPaths("G:/OneDrive/Documentos/Doutorado/protocol_database/libraires")                 #
#                                                                                          #
############################################################################################


# Carregue a biblioteca doParallel
library(doParallel)

# Configure o cluster paralelo (pode usar o número de núcleos desejado)
cl <- makeCluster(8)
registerDoParallel(cl)

library(caret)
############################################################################################

# Reading CSV files
setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")

############################################################################################

DB1 <- read.csv("DB1.csv", header=TRUE, sep=",", dec=".", na.strings="NA", check.names=FALSE)
DB_train <- read.csv("DB_train.csv", header=TRUE, sep=",", dec=".", na.strings="NA", check.names=FALSE)
atr_content <- read.csv("atr_content.csv", header=TRUE, sep=",", dec=".", na.strings="NA", check.names=FALSE)

# Converting atr_content to numeric
atr_content <- as.numeric(unlist(atr_content))

# CHOOSE HOW MANY TIMES TO RUN
K <- 50

# CREATE SPREADSHEETS
predictions <- matrix(nrow = length(atr_content), ncol = K)
stat_pred_cv <- matrix(nrow = 3, ncol = K)

# Get the number of columns in DB_train
num_columns <- ncol(DB_train)
impcov <- matrix(ncol = K, nrow = num_columns)
############################################################################################

# Define the column name of the dependent variable
depend_var <- "C_gkg"

############################################################################################

# Loop through iterations
for (i in 1:K) {

  # Check and install required packages
  if (!require(parallel)) {install.packages("parallel"); require(parallel)}
  detectCores()

  if (!require(doParallel)) {install.packages("doParallel"); require(doParallel)}

  # Set up parallel processing
  cl <- makePSOCKcluster(8) # Choose the number of CPU cores
  registerDoParallel(cl)

  # Create a new dataframe DB_IDs that includes a column with the indices of the rows
  DB_IDs <- cbind(Row_Index = rownames(DB1), DB1)


  # Create 10 folds
  folds <- createFolds(DB1[[depend_var]], k = 10)

  ctrl <- trainControl(method = "cv", index = folds)

  # Sample rows for training
  cal_rows <- sample(nrow(DB_train), nrow(DB_train), replace = TRUE)

#########################################################################################
# Enter the name of the dependent covariate, matching the name in the data sheet column.

    Atr_Cub_Model <- train(x = DB_train[cal_rows, ],
                           y = DB1[[depend_var]][cal_rows],
                           method = "cubist",
                           trControl = ctrl)
    print(Atr_Cub_Model)

#########################################################################################

  ###############################################################################################################
  # Custom commands to obtain the average RPIQ of the best model
  ###############################################################################################################

  # Obtain the order of folds in the model output
  fold_order <- Atr_Cub_Model[["resample"]][["Resample"]]

  # Obtain Rsquared and RMSE values from the model output
  rsquared_values <- Atr_Cub_Model[["resample"]][["Rsquared"]]
  rmse_values <- Atr_Cub_Model[["resample"]][["RMSE"]]

  # Create new vectors of Rsquared and RMSE corresponding to the original fold order
  rsquared_ordered <- rep(NA, length(folds))
  rmse_ordered <- rep(NA, length(folds))

  for (m in 1:length(fold_order)) {
    fold_number <- as.numeric(sub("Fold", "", fold_order[m]))
    rsquared_ordered[fold_number] <- rsquared_values[m]
    rmse_ordered[fold_number] <- rmse_values[m]
  }

  # Add Rsquared and RMSE to the folds list
  for (m in 1:length(folds)) {
    folds[[m]]$Value_Rsquared <- rsquared_ordered[m]
    folds[[m]]$Value_RMSE <- rmse_ordered[m]
  }

  # Initialize the folds_IDs list
  folds_IDs <- list()

  # Fill the list with IDs and dependent variable values for each fold
  for (m in 1:length(folds)) {
    fold_indices <- unlist(folds[[m]])
    # Use match to find the corresponding indices in DB_IDs based on Row_Index values
    matched_indices <- match(fold_indices, DB_IDs$Row_Index)
    fold_data <- DB_IDs[matched_indices, c("Row_Index","ID_Unico", depend_var)]
    folds_IDs[[m]] <- fold_data
  }

  # Initialize lists to store first quartiles and third quartiles
  first_quartiles <- vector("numeric", length(folds_IDs))
  third_quartiles <- vector("numeric", length(folds_IDs))

  # Calculate median and third quartile for each fold
  for (m in 1:length(folds_IDs)) {
    fold_data <- folds_IDs[[m]][[depend_var]]
    first_quartiles[m] <- quantile(fold_data, probs = 0.25, na.rm = TRUE)  # 1st quartile
    third_quartiles[m] <- quantile(fold_data, probs = 0.75, na.rm = TRUE)  # 3rd quartile
  }

  # Add medians and third quartiles to the folds list
  for (m in 1:length(folds)) {
    folds[[m]]$First_Quartile <- first_quartiles[[m]]
    folds[[m]]$Third_Quartile <- third_quartiles[[m]]
  }

  # Calculate Value_RPIQ for each fold
  for (m in 1:length(folds)) {
    folds[[m]]$Value_RPIQ <- (folds[[m]]$Third_Quartile - folds[[m]]$First_Quartile) / folds[[m]]$Value_RMSE
  }

  # Initialize a list to store the RPIQ values
  rpiq_values <- vector("numeric", length(folds))

  # Store the RPIQ value of each fold in the list
  for (m in 1:length(folds)) {
    rpiq_values[m] <- folds[[m]]$Value_RPIQ
  }

  # Print RPIQ values for each fold
  for (m in 1:length(folds)) {
    cat("Fold", m, "RPIQ:", folds[[m]]$Value_RPIQ, "\n")
  }

  ###############################################################################################################

  # Generate a .csv file for each fold
  # for (i in 1:length(folds_IDs)) {
  #   write.csv(folds_IDs[[i]], file = paste0("Fold_", i, ".csv"))
  # }

  # Print the model summary
  print(Atr_Cub_Model)

  print(Atr_Cub_Model[["bestTune"]])

  Atr_Cub_Model[["resample"]][["Resample"]]

  Atr_Cub_Model[["resample"]][["Rsquared"]]

  Atr_Cub_Model[["resample"]][["RMSE"]]

  # Calculate the mean of RPIQ values
  average_rpiq <- mean(rpiq_values, na.rm = TRUE)

  # Print the mean of RPIQ values
  cat("The mean of RPIQ values is:", average_rpiq, "\n")

  # Extract and order cross-validation results
  CV <- as.data.frame(cbind(Atr_Cub_Model$results$Rsquared, Atr_Cub_Model$results$RMSE))
  colnames(CV) <- c("R2","RMSE")
  ordenadoCV <- as.data.frame(CV[order(CV$RMSE, decreasing = FALSE), ])[1,]

  R2 <- ordenadoCV$R2
  RMSE <- ordenadoCV$RMSE
  RPIQ <- average_rpiq

  restusCV_1 <- rbind(R2, RMSE, RPIQ)

  stat_pred_cv[, i] <- restusCV_1

  # Covariables importance
  ImpCov <- varImp(Atr_Cub_Model)
  ImpCoV1 <- as.data.frame(ImpCov$importance)
  write.csv(ImpCoV1, "VAR.csv")
  ImpCoV2 <- read.csv("VAR.csv")
  ImpCoV3 <- ImpCoV2[order(ImpCoV2$X, decreasing = FALSE), ]
  rownames(ImpCoV3) <- ImpCoV3$X
  ImpCoV4 <- ImpCoV3$Overall

  # Predictions
  pred_Cub <- predict(Atr_Cub_Model, DB_train)

  #ID_Unico <- as.numeric(DB1$ID_Unico)
  ID_Unico <- as.character(DB1$ID_Unico)  # Ensure ID_Unico is character

  Predict <- as.data.frame(cbind(ID_Unico, atr_content, pred_Cub))

  pred_Cub2 <- data.frame(pred_Cub)
  predictions[, i] <- pred_Cub2[, ncol(pred_Cub2)]
  impcov[, i] <- t(ImpCoV4)

  # Stop parallel processing
  stopCluster(cl)
}

impcov <- as.data.frame(impcov)
impcov$X <- ImpCoV3$X
impcov <- impcov[, c("X", setdiff(names(impcov), "X"))]

############################################################################################

setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")
write.csv(impcov, "ImpCoV_DB_filter_step_5_Clay.csv")
write.csv(stat_pred_cv, "acuracia_DB_filter_step_5_Clay.csv")

############################################################################################

### Statistical analysis of predictions ###

# Calculate mean prediction for each row
mean_pred <- apply(predictions, 1, mean)

# Calculate standard deviation for each row
sd_pred <- apply(predictions, 1, sd)

# Coefficient of Variation (CV) function
cv <- function(x) {
  sd(x) / mean(x) * 100
}

# Calculate CV for each row
cv_pred <- apply(predictions, 1, cv)

# Calculate 90% prediction interval for each row
interval_T <- t(apply(predictions, 1,
                      function(x) {
                        quantile(x,
                                 probs = c(0.05, 0.95),
                                 na.rm = TRUE)
                      }))

# Extract lower and upper bounds of the prediction interval
Lower_T <- interval_T[, 1]
Upper_T <- interval_T[, 2]

# Calculate the width of the 90% prediction interval
interval_90_PI_pred <- Upper_T - Lower_T

# Create a dataframe with all statistical parameters
stat_p <- data.frame(cbind(ID_Unico, atr_content, predictions, mean_pred, cv_pred, sd_pred, Lower_T, Upper_T, interval_90_PI_pred))

############################################################################################

# Write the dataframe to a CSV file
setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")
write.csv(stat_p, "stat_p.csv")

############################################################################################

"""

# Executing the R script
robjects.r(r_script)


## Cutting line determination

In [ ]:
# Packages required
import os
import statsmodels.api as sm
import numpy as np
import pandas as pd
from plotnine import *
import scipy.stats as stats
from plotnine import ggplot

In [ ]:
# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")

# Read the CSV file
stat_p = pd.read_csv("stat_p.csv")

print(stat_p)

In [ ]:
### Cutting line determination ###

# Specify the X and Y columns
column_x = stat_p['atr_content']
column_y = stat_p['mean_pred']

# Create a linear regression model
X = sm.add_constant(column_x)  # Add the constant for the intercept term
model = sm.OLS(column_y, X).fit()

# Calculate predicted values
predictions = model.predict(X)

# Calculate the standard deviation of residuals
residuals = model.resid
std_residuals = np.std(residuals)

# Confidence level (e.g., 98%)
confidence_level = 0.99

# Quantile of the t distribution
quantile_t = stats.t.ppf((1 + confidence_level) / 2, df=len(residuals) - 2)

# Calculate upper and lower confidence limits
upper_limit_attribute = predictions + quantile_t * (std_residuals * 1.15)
lower_limit_attribute = predictions - quantile_t * (std_residuals * 1.15)

# Regression coefficients
intercept, coef_x = model.params

# Standard error of coefficients
std_error = model.bse

# T-value and p-value of coefficients
t_value = model.tvalues
p_value = model.pvalues

# Coefficient of determination (R²)
r_squared_simulated = model.rsquared

# Fit a linear regression model for the upper limit
model_upper_limit = sm.OLS(upper_limit_attribute, X).fit()

# Fit a linear regression model for the lower limit
model_lower_limit = sm.OLS(lower_limit_attribute, X).fit()

# Coefficients of intercept and slope for the upper limit
coef_intercept_upper, coef_slope_upper = model_upper_limit.params

# Coefficients of intercept and slope for the lower limit
coef_intercept_lower, coef_slope_lower = model_lower_limit.params

# Display coefficients
print("Coefficients for Upper Limit:")
print("Intercept (β0):", round(coef_intercept_upper, 4))
print("Slope (β1):", round(coef_slope_upper, 4))

print("\nCoefficients for Lower Limit:")
print("Intercept (β0):", round(coef_intercept_lower, 4))
print("Slope (β1):", round(coef_slope_lower, 4))

In [ ]:
### Observed clay vs. predicted clay scatterplot ###

# Desired width and height of the plot
plot_width = 9
plot_height = 6

scatter_plot = (
    ggplot(stat_p, aes(x='atr_content', y='mean_pred')) +
    theme_classic() +
    geom_point(shape='o', color= 'black', fill= '#8069C7', alpha=1, size=1.5, stroke=0.2) + # Data points with black contour +  # Data points
    geom_smooth(method='lm', se=False, color='#000000', size=1.2) +  # Trend line
    geom_line(aes(y=upper_limit_attribute), color='#E41A1C', size=1) +  # Upper confidence interval line
    geom_line(aes(y=lower_limit_attribute), color='#E41A1C', size=1) +  # Lower confidence interval line

    annotate('text', x=55, y=232, label="β upper limit: {:.3f}".format(round(coef_intercept_upper, 3)),
             family="Arial", size=9)+
    annotate('text', x=55, y=220, label="β lower limit: {:.3f}".format(round(coef_intercept_lower, 3)),
             family="Arial", size=9)+
    annotate('text', x=55, y=208, label="R² model: {:.3f}".format(round(r_squared_simulated, 3)),
             family="Arial", size=9)+
    annotate('text', x=55, y=198, label="α: {:.3f}".format(round(coef_x, 3)),
             family="Arial", size=9)+

    #labs(x="Observed clay content (g kg⁻¹)", y="Predicted clay content (g kg⁻¹)") +
    labs(x="Observed C content (g kg$^{-1}$)", y="Predicted C by SySI content (g kg$^{-1}$)") +
    expand_limits(x=0, y=0) +  # Force the plot to show the origin (0,0)
    scale_y_continuous(limits=[0, 250]) +
    scale_x_continuous(limits=[0, 250]) +
    theme(
        axis_title_x=element_text(margin={'t': 12}, family="Arial", size=14, face="bold"),
        axis_title_y=element_text(margin={'r': 12}, family="Arial", size=14, face="bold"),
        axis_text_y=element_text(family="Arial", size=12),
        axis_text_x=element_text(family="Arial", size=12)
    )
)

# Display the plot
print(scatter_plot)

dpi_value = 1000

# Setting the working directory
output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon/scatter_plot_C_predicted_by_SySI.png"
scatter_plot.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

In [ ]:
### After the agreement analysis, filtering the samples in the database ###

# Create a subset of samples that are ABOVE the upper_limit_attribute line
samples_upper_limit = stat_p[stat_p['mean_pred'] > upper_limit_attribute]

# Create a subset of samples that are BELOW the lower_limit_attribute line
samples_lower_limit = stat_p[stat_p['mean_pred'] < lower_limit_attribute]

# Concatenate the dataframes samples_upper_limit and samples_lower_limit
outliers_samples = pd.concat([samples_upper_limit, samples_lower_limit])

# Merge the DataFrames based on the ID_Unico column and keep only the columns from the DB DataFrame
outliers_samples_6 = pd.merge(outliers_samples[['ID_Unico']], DB, on='ID_Unico', how='inner')

IDs_to_filter = outliers_samples['ID_Unico'].tolist()

DB_filter_step_6 = DB_filter_step_5

# Add a new column "outliers_step_4" filled with False
DB_filter_step_6.insert(1, "outliers_step_6", False)

# Update the values to True where ID_Unico is in the list IDs_to_filter
DB_filter_step_6.loc[DB_filter_step_6['ID_Unico'].isin(IDs_to_filter), 'outliers_step_6'] = True

# Create a new DataFrame with only the rows where outliers_step_4 is True
outliers_samples_step_6 = DB_filter_step_6[DB_filter_step_6['outliers_step_6'] == True].copy()

# Remove the samples where outliers_step_6 is True
DB_filter_step_6 = DB_filter_step_6[DB_filter_step_6['outliers_step_6'] == False].copy()

print(outliers_samples_step_6)


In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_6, and DB_filter_step_6 dataframe

# Print the number of samples in DB_filter_step_5
print("Number of samples in DB_filter_step_5:", DB_filter_step_5.shape[0])

# Print the number of samples in outliers_samples_step_6
print("Number of samples in outliers_samples_step_6:", outliers_samples_step_6.shape[0])

# Print the number of samples in DB_filter_step_6
print("Number of samples in DB_filter_step_6:", DB_filter_step_6.shape[0])

In [ ]:
# Save the DB_filter_step_6 and outliers_samples_step_6 DataFrame as a CSV file
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/Step_6_SySI_data/Organic_Carbon")

# Save the outliers_samples_6 DataFrame as a CSV file
outliers_samples_step_6.to_csv("outliers_samples_multispectraldata_step_6_C.csv", index=False)

# Save the DB_filter_6 DataFrame as a CSV file
DB_filter_step_6.to_csv("DB_filter_multispectraldata_step_6_C.csv", index=False)

# **Analysis of database variability**

In [ ]:
# Packages required
import os
import pandas as pd
import folium
from folium import raster_layers
import ipywidgets as widgets
from IPython.display import display
import plotnine as p9
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# Set the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/Step_3")

# Read the CSV file
df1 = pd.read_csv("outliers_samples_step_3.csv")

os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/Step_4")

df2=pd.read_csv("outliers_samples_step_4.csv")

In [ ]:
print(df2)

In [ ]:
import folium
from folium import raster_layers
import ipywidgets as widgets
from IPython.display import display

# Supondo que 'df1' e 'df2' são os seus DataFrames e 'X' e 'Y' são as colunas de longitude e latitude
mapa1 = folium.Map(location=[df1['Y'].mean(), df1['X'].mean()], zoom_start=3, control_scale=True)
mapa2 = folium.Map(location=[df2['Y'].mean(), df2['X'].mean()], zoom_start=3, control_scale=True)

# Adiciona o mapa de satélite do Google aos dois mapas
for mapa in [mapa1, mapa2]:
    raster_layers.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = False,
        control = True
    ).add_to(mapa)

# Adiciona marcadores ao mapa1 usando os dados em df1
for _, row in df1.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=0.8,
        color="#f44141",
        fill=True,
        fill_color="#f44141",
        fill_opacity=1
    ).add_to(mapa1)

# Adiciona marcadores ao mapa2 usando os dados em df2
for _, row in df2.iterrows():
    folium.CircleMarker(
        location=[row['Y'], row['X']],
        radius=0.8,
        color="#148fae",
        fill=True,
        fill_color="#148fae",
        fill_opacity=1
    ).add_to(mapa2)

# Adiciona controles de camada aos dois mapas
folium.LayerControl().add_to(mapa1)
folium.LayerControl().add_to(mapa2)

# Cria widgets de saída para exibir os mapas
out1 = widgets.Output()
out2 = widgets.Output()

# Exibe os mapas dentro dos widgets de saída
with out1:
    display(mapa1)
with out2:
    display(mapa2)

# Exibe os widgets de saída lado a lado usando ipywidgets.HBox()
widgets.HBox([out1, out2])

In [ ]:
# Cria o histograma
grafico1 = (p9.ggplot(df1, p9.aes(x='Clay_gkg'))
           + p9.geom_histogram(fill='#f44141', binwidth=8)  # Controla a cor das barras
           + p9.scale_x_continuous(limits=[0, 900])  # Controla a escala do eixo x
           + p9.scale_y_continuous(limits=[0, 50])  # Controla a escala do eixo y
          )

# Cria o histograma
grafico2 = (p9.ggplot(df2, p9.aes(x='Clay_gkg'))
           + p9.geom_histogram(fill='#148fae', binwidth=8)  # Controla a cor das barras
           + p9.scale_x_continuous(limits=[0, 900])  # Controla a escala do eixo x
           + p9.scale_y_continuous(limits=[0, 50])  # Controla a escala do eixo y
          )

# Salva os gráficos como imagens com alta resolução
grafico1.save("grafico1.png", dpi=800)
grafico2.save("grafico2.png", dpi=800)

# Cria uma nova figura com tamanho maior
plt.figure(figsize=(15,5))

# Adiciona o primeiro subplot
plt.subplot(1,2,1) # (nrows, ncols, index)
img = mpimg.imread('grafico1.png')
imgplot = plt.imshow(img)
plt.axis('off')  # Não mostra os eixos

# Adiciona o segundo subplot
plt.subplot(1,2,2)
img = mpimg.imread('grafico2.png')
imgplot = plt.imshow(img)
plt.axis('off')  # Não mostra os eixos

plt.show()

# **Clear all variables**

In [ ]:
# Remove todas as variáveis do ambiente
%reset -f

# Remove todas as imports e módulos carregados
import gc
gc.collect()